In [42]:
import ophion
import json
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas


In [2]:
%matplotlib inline

In [3]:
O = ophion.Ophion("http://bmeg.compbio.ohsu.edu")

In [4]:
#get the list of drugs
drugs = list(set(O.query().has("gid", "cohort:CCLE").outgoing("hasMember").incoming("responseOf").outgoing("responseTo").values(["gid"]).execute()['result']))
#print drugs

[u'compound:JQ-1:MK-0752 (1:1 mol/mol)', u'compound:A-804598', u'compound:CIL55A', u'compound:I-BET151', u'compound:SNX-2112', u'compound:NSC95397', u'compound:palmostatin B', u'compound:DBeQ', u'compound:JW-55', u'compound:tivantinib', u'compound:Compound 7d-cis', u'compound:importazole', u'compound:pifithrin-mu', u'compound:ISOX:bortezomib (250:1 mol/mol)', u'compound:BRD-K48477130', u'compound:SR8278', u'compound:sitagliptin', u'compound:QW-BI-011', u'compound:tipifarnib-P1', u'compound:tipifarnib-P2', u'compound:tanespimycin:bortezomib (250:1 mol/mol)', u'compound:salermide', u'compound:BRD-A05715709', u'compound:belinostat', u'compound:dinaciclib', u'compound:ibrutinib', u'compound:CID-5951923', u'compound:BRD-K19103580', u'compound:BRD4132', u'compound:BRD-K58730230', u'compound:UNC0638:navitoclax (1:1 mol/mol)', u'compound:cediranib', u'compound:marinopyrrole A', u'compound:necrosulfonamide', u'compound:HC-067047', u'compound:CD-1530', u'compound:KPT185', u'compound:GSK461364', 

In [10]:
all_samples = O.query().has("gid", "cohort:CCLE").outgoing("hasMember").mark("a").incoming("callsFor").select("a").values(["gid"]).execute()['result']

In [47]:
GENE = "MDM2"

In [48]:
mut_samples = O.query().has("gid", "gene:%s" % (GENE)).incoming("affectsGene").incoming("transcriptEffectOf").outgoing("annotationFor").\
outgoing("inCallSet").outgoing("callsFor").mark("a").incoming("hasMember").has("gid", "cohort:CCLE").select("a").values(["gid"]).execute()['result']

norm_samples = list(set(all_samples).difference(mut_samples))

In [49]:
def get_set_drug_summaries(samples, drug, responseType):
    #data = O.query().has("gid", samples).incoming("responseOf").mark("a").outgoing("responseTo").\
#has("gid", drug).select("a").values(["gid", "responseSummary"]).execute()['result']
    data = O.query().has("gid", drug).incoming("responseTo").mark("a").outgoing("responseOf").has("gid", samples).select("a").values(["gid", "responseSummary"]).execute()['result']
    data_map = dict(list( (data[i], json.loads(data[i+1])) for i in range(0, len(data), 2) ))
    #print data_map
    values = list( filter(lambda x:x["type"] == responseType, i) for i in data_map.values())
    values = filter(lambda x: len(x) > 0, values)
    values = map(lambda x:x[0]['value'], values)
    return values

In [50]:
out = []
for drug in drugs:
    mut_values = get_set_drug_summaries(mut_samples, drug, "AUC")
    norm_values = get_set_drug_summaries(norm_samples, drug, "AUC")
    s = stats.ttest_ind(mut_values, norm_values, equal_var=False)
    out.append( {"drug" : drug, "statistic" : s.statistic, "pvalue" : s.pvalue} )

/usr/local/lib/python2.7/site-packages/numpy/core/_methods.py:82: RuntimeWarning: Degrees of freedom <= 0 for slice
  warnings.warn("Degrees of freedom <= 0 for slice", RuntimeWarning)
/usr/local/lib/python2.7/site-packages/numpy/core/_methods.py:116: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [51]:
pandas.DataFrame(out).sort_values("statistic")

,drug,pvalue,statistic
184,compound:curcumin,0.000491,-3.889820
16,compound:sitagliptin,0.007226,-3.288333
435,compound:gefitinib,0.005188,-3.050099
205,compound:narciclasine,0.006262,-2.951526
264,compound:BRD-K13185470,0.014514,-2.720369
526,compound:BRD-K75293299,0.021946,-2.718516
265,compound:BRD-K17060750,0.024449,-2.384916
352,compound:BRD-K34485477,0.032522,-2.368003
207,compound:WAY-362450,0.031534,-2.275235
110,compound:BRD-K52037352,0.032898,-2.254667


In [19]:
drug="compound:968"
print O.query().has("gid", mut_samples).incoming("responseOf").mark("a").outgoing("responseTo").\
has("gid", drug).select("a").values(["gid", "responseSummary"]).execute()['result']

[u'responseCurve:HT29_LARGE_INTESTINE:968', u'[{"type":"EC50","value":0.026139999999999997,"unit":"uM"},{"type":"AUC","value":14.879,"unit":"uM"}]', u'responseCurve:RL952_ENDOMETRIUM:968', u'[{"type":"EC50","value":0.338,"unit":"uM"},{"type":"AUC","value":14.492,"unit":"uM"}]', u'responseCurve:ISHIKAWAHERAKLIO02ER_ENDOMETRIUM:968', u'[{"type":"EC50","value":30.59,"unit":"uM"},{"type":"AUC","value":15.086,"unit":"uM"}]', u'responseCurve:HCC1588_LUNG:968', u'[{"type":"EC50","value":5.658E-15,"unit":"uM"},{"type":"AUC","value":13.986,"unit":"uM"}]', u'responseCurve:LS513_LARGE_INTESTINE:968', u'[{"type":"EC50","value":64.52,"unit":"uM"},{"type":"AUC","value":13.43,"unit":"uM"}]', u'responseCurve:HEC1B_ENDOMETRIUM:968', u'[{"type":"EC50","value":0.1495,"unit":"uM"},{"type":"AUC","value":14.467,"unit":"uM"}]', u'responseCurve:22RV1_PROSTATE:968', u'[{"type":"EC50","value":0.03288,"unit":"uM"},{"type":"AUC","value":14.376,"unit":"uM"}]', u'responseCurve:OUMS23_LARGE_INTESTINE:968', u'[{"type

In [27]:
print O.query().has("gid", drug).incoming("responseTo").mark("a").outgoing("responseOf").has("gid", mut_samples).select("a").values(["gid", "responseSummary"]).execute()

{u'result': [u'responseCurve:HPBALL_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE:A-804598', u'[{"type":"EC50","value":2.374E-17,"unit":"uM"},{"type":"AUC","value":15.326,"unit":"uM"}]', u'responseCurve:RKO_LARGE_INTESTINE:A-804598', u'[{"type":"EC50","value":60.85,"unit":"uM"},{"type":"AUC","value":14.635,"unit":"uM"}]', u'responseCurve:MDAMB231_BREAST:A-804598', u'[{"type":"EC50","unit":"uM"},{"type":"AUC","value":15.199,"unit":"uM"}]', u'responseCurve:SNUC5_LARGE_INTESTINE:A-804598', u'[{"type":"EC50","value":5.165E10,"unit":"uM"},{"type":"AUC","value":15.0,"unit":"uM"}]', u'responseCurve:HEYA8_OVARY:A-804598', u'[{"type":"EC50","value":0.5155,"unit":"uM"},{"type":"AUC","value":14.908,"unit":"uM"}]', u'responseCurve:A375_SKIN:A-804598', u'[{"type":"EC50","value":112800.0,"unit":"uM"},{"type":"AUC","value":15.0,"unit":"uM"}]', u'responseCurve:WM793_SKIN:A-804598', u'[{"type":"EC50","value":0.4424,"unit":"uM"},{"type":"AUC","value":15.0,"unit":"uM"}]', u'responseCurve:GP2D_LARGE_INTESTINE:A-8045